# 4.1. Інтерполяція функцій поліномами
--------------------

Інтерполювання функції  $y=f(x),\, x\in [a,b],$ поліномами 

$(1)\quad\qquad L_n (x)= \sum_{i=0}^n a_i x^i, \; n\in \mathbb{N}\cup \{0\},$

полягає у знаходженні таких поліномів, які наближують цю функцію на відрізку $[a,b]$ з деякою точністю, 
якщо відомі значення $ f_0 = f(x_0),\, f_1 = f(x_1), ... ,\, f_n = f(x_n)$ на скінченій множині точок $ x_0, x_1, ... , x_n$. 

## 4.1.2. Інтерполяційний поліном Ньютона
-----------
Інтерполяційний поліном Ньютона $L_n(x)$ є такою формою подання інтерполяційного полінома, яка забезпечує його обчислення через відповідне значення $L_{n-1}(x)$:

$(5)\quad\qquad L_n(x) = L_{n-1}(x) + (x-x_0)(x-x_1) \ldots (x-x_{n-1})f(x_0;x_1; \ldots; x_n), \; n \in  \mathbb{N},$


де $f(x_0;x_1; \ldots; x_n)$ -- розділена різниця $n$-го порядку.

Розділені різниці обчислюють рекурентно за формулю

$(5)\quad\qquad
f(x_{i-1};x_{i}; \ldots; x_{i+k}):=\dfrac{f(x_{i};x_{i+1}; \ldots; x_{i+k}) - f(x_{i-1};x_{i}; \ldots;
x_{i+k-1})}{x_{i+k}-x_{i-1}},\; i=\overline{1, n-k},\; k=\overline{1, n}.$

Під час обчислення розділені різниці можна зберігати у такій структурі з рядками змінної довжини

\begin{equation}
\begin{cases}
f(x_0) \\
f(x_1) \quad f(x_0; x_1) \\
f(x_2) \quad f(x_1; x_2) \quad f(x_0; x_1; x_2)\\
 \ldots\quad \quad \ldots  \\
f(x_n) \quad f(x_{n-1}; x_n)  \quad f(x_{n-2}; x_{n-1}; x_n) \quad \ldots \quad f(x_0;x_1; \ldots; x_n),
\end{cases}
\end{equation}

У явному вигляді подання (5) можна переписати так

$ L_n(x)= f(x_0)+(x-x_0)f(x_0;x_1) + (x-x_0)(x-x_1)f(x_0;x_1;x_2) + \ldots +(x-x_0)(x-x_1) \ldots (x-x_{n-1}) f(x_0;x_1;\ldots; x_n).
$

#### Пояснення до використання програмного коду
-----------------
*   Підготувати середовище і потрібні функції : 
    1. виконати комірку для підготовки середовища 
    2.   виконати комірки, де **визначені** функції ``divided_differences``,``Newton_interpolation`` і ``N_interpolator``

*   Обчислити наближені значення конкретної функції :   :    
    1. виконати комірку, де **визначені** функції, які задають вузли інтерполювання і значення конкретної функції у цих вузлах
    2. виконати комірку, в якій задають координати відрізка ``[a,b]`` і викликають функції для задання даних інтерполювання і обчислення розділених різниць
    3. якщо треба обчислити: 
        -  одне наближене значення функції в деякій точці, то виконати комірку з викликом функції ``Newton_interpolation`` з відповідними значеннями аргументів
        -  кілька наближених значень функції в рівновіддалених точках на ``[a,b]``, то викликати функцію ``N_interpolator`` з відповідними значеннями аргументів; у цьому випадку за замовчуванням (при ``prnt=True`` ) будуть побудовані графіки полінома і функції, яку інтерполюють (при ``fr=True`` ), а також буде обчислено найбільше відхилення знайденого полінома від цієї фукції.

#### Програмна реалізація методу
------------

>#### Підготовка середовища

In [1]:
#%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

>#### ``divided_differences`` -- функція для обчислення розділених різниць функції ``f`` на масиві точок ``xs``

In [2]:
def divided_differences(xs,f):
    """обчислення розділених різниць функції f на масиві точок xs """
    ddm=np.empty(xs.size)
    ddm[0]=f[0]
    
    dd=list()
    dd.append(f)   
    n=xs.size-1
    
    for k in range(1,n+1):
        fn=np.empty(n-k+1)
        for i in range(n-k+1):            
            fn[i]=(dd[k-1][i+1]-dd[k-1][i])/(xs[i+k]-xs[i])
        ddm[k]=fn[0]
        dd.append(fn)
    return ddm      

>#### ``Newton_interpolation`` -- функція для обчислення інтерполяційного полінома Ньютона

In [3]:
def Newton_interpolation(x,k, xs, ddm):
    """обчислення значення інтерполяційного полінома 
       x - точка, в якій обчислюють значення полінома
       k - степінь полінома
       xs - масив вузлів
       ddm - розділені різниці
    """
    pk=ddm[0]
    w=1
    for i in range(1,k+1):
        w*=x-xs[i-1]
        pk+=w*ddm[i]       
    return pk

>#### ``N_interpolator`` -- функція, яка реалізує процес інтерполювання

In [4]:
def N_interpolator(xv,fv,a,b,n,m,ng,prnt=True,fr=False):
    """наближення заданої таблично функції на відрізку [a,b] поліномом Ньютона m-го степеня
       xv - функція, яка встановлює вузли інтерполювання
       fv - функція, яка задає табличні значення функції у вузлах інтерполювання
       ng - кількість точок, у яких обчислюють значення полінома
    """
    x=xv(a,b,n)
    
    if m>x.size-1:
        print(f" недостатня кількість інтерполяційних вузлів")
        return
    
    f=fv(a,b,n)
    ddm=divided_differences(x,f)
    
    xg=np.linspace(a,b,ng+1)

    pm=np.empty(ng+1)
    i=0
    for xi in xg:
        pm[i]=Newton_interpolation(xi,m, x, ddm)
        i+=1
    
    if prnt== False:
        return pm
    
    plt.close('all')
    
    fig = plt.figure(figsize=(8, 5))
    ax = fig.gca()
    ax.axhline(color="grey", ls="--", zorder=-1)
    ax.axvline(color="grey", ls="--", zorder=-1)
    plt.plot(xg, pm)
    
    plt.scatter(x, f, marker='o')
    if fr==True:
        fg=fv(a,b,ng)
        plt.plot(xg, fg,'--')
        eps=np.max(np.abs(pm-fg))
        print(f"eps={eps}")
    
    return pm    

#### Обчислювальні експерименти
____________________

Продемонструємо використання інтерполяційного полінома Ньютона для наближення функцій.

**Приклад 1.** Інтерполювання функції $f$, яка на множині $(0, 2, 3, 5)$ набуває значення $f(0)=2, \, f(2)=4,\, f(3)=6, \,f(5)=8$. 


**Приклад 1.** Інтерполювання функції, заданої таблицею   

 $$
 \begin{tabular}{|c|c|c|c|c|c|}
 \hline
 $i$ & $0$ & $1$ & $2$ & $3$
 \\
 \hline
 $x_i$ & $0$ & $2$ & $3$ & $5$
 \\ \hline
 $y_i $ %\phantom{$\Bigg|$}
 & $2$ & $4$ & $6$  & $8$
  \\ \hline
 \end{tabular} 
 
 $$
 
 на відрізку $[a, b]$

Визначимо функції, які повертають вузли інтерполювання та відповідні значення функції:

In [5]:
def xv1(a,b,n):    
    """ встановлення вузлів інтерполювання"""
    return np.array([0,2,3,5], dtype='float32')

In [6]:
def fv1(a,b,n):
    """ задання значень функції у вузлах інтерполювання"""
    return np.array([2,4,6,8], dtype='float32')

Задамо також кінці відрізка та кількість вузлів інтерполювання:

In [7]:
a=0
b=5
n=3

Якщо треба обчислити наближені значення функції **в окремих точках**, то спочатку обчислимо за відомими значеннями функцій розділені різниці:

In [8]:
x1=xv1(a,b,n)
ddm = divided_differences(x1,fv1(a,b,n))

і використаємо їх у функції ``Newton_interpolation`` для побудови інтерполяційного полінома Ньютона ``m``-го степеня (``m <= n``) та обчислення його значення в точці ``xi``: 

In [9]:
xi=1.5
m=3
Newton_interpolation(xi,m, x1, ddm)

3.1

Щоб знайти значення інтерполяційного полінома в іншій точці, задаємо її координати і повторюємо виклик ``Newton_interpolation`` :

In [10]:
xi=2.5
Newton_interpolation(xi,m, x1, ddm)

5.0

Якщо ж треба обчислити значення інтерполяційного полінома Ньютона ``m``-го степеня в ``ng`` **рівновіддалених точках** на ``[a,b]``, то використовуємо функцію ``N_interpolator``, яка визначає необхідні для інтерполювання кроки
:

In [11]:
m=3
ng=60
N_interpolator(xv1,fv1,a,b,n,m,ng,prnt=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([2.        , 1.9679784 , 1.94938272, 1.94375   , 1.95061728,
       1.9695216 , 2.        , 2.04158951, 2.09382716, 2.15625   ,
       2.22839506, 2.30979938, 2.4       , 2.49853395, 2.60493827,
       2.71875   , 2.83950617, 2.96674383, 3.1       , 3.23881173,
       3.38271605, 3.53125   , 3.68395062, 3.84035494, 4.        ,
       4.16242284, 4.32716049, 4.49375   , 4.6617284 , 4.83063272,
       5.        , 5.16936728, 5.3382716 , 5.50625   , 5.67283951,
       5.83757716, 6.        , 6.15964506, 6.31604938, 6.46875   ,
       6.61728395, 6.76118827, 6.9       , 7.03325617, 7.16049383,
       7.28125   , 7.39506173, 7.50146605, 7.6       , 7.69020062,
       7.77160494, 7.84375   , 7.90617284, 7.95841049, 8.        ,
       8.0304784 , 8.04938272, 8.05625   , 8.05061728, 8.0320216 ,
       8.        ])

Зазначимо, що функція ``N_interpolator`` при значеннях аргументів ``prnt=True`` виводить на графічну панель графік інтерполяційного полінома, а якщо відома формула інтерпольованої функції (як це буде продемонстровано в наступному прикладі), то при ``fr=True`` будується графік цієї функції і виводиться максимум відхилення інтерполяційного полінома від заданої функції.

**Приклад 2.** Інтерполювання функції $f(x)=sin(x),\; x \in [0,2\pi]$  на **рівновіддалених вузлах**. 

Визначимо функції, які окремо повертають множину рівновіддалених вузлів інтерполювання та відповідні значення функції:

In [12]:
def xv2(a,b,n):
    x=np.linspace(a,b,n+1)
    return x

In [13]:
def fv2(a,b,n):   
    x=np.linspace(a,b,n+1)
    f=np.sin(x)#+1
    return f

Визначимо координати відрізка інтерполювання та кількість вузлів:

In [14]:
a=0
b=2*np.pi
n=3

Тепер обчислимо значення інтерполяційного полінома Ньютона ``m``-го степеня в ``ng`` точках відрізка:

In [15]:
m=3
ng=60
pN = N_interpolator(xv2,fv2,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=0.2545937399527193


Розглянемо тепер інтерполяцію поліномами вищого степеня. Як і слід очікувати, при заданні степеня полінома меншим кількості використаних при цьому вузлів матимемо значне відхилення значень полінома від значень функції, наприклад:    

In [16]:
n = 5
m = 3
pN = N_interpolator(xv2,fv2,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=3.3677098243468913


Тому далі, збільшуючи кількість вузлів, братимемо такий самий степінь полінома: 

In [17]:
n = 5
m = 5
pN = N_interpolator(xv2,fv2,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=0.02664282649990979


In [18]:
n = 10
m = 10
pN = N_interpolator(xv2,fv2,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=5.0895173235671054e-05


In [19]:
n = 20
m = 20
pN = N_interpolator(xv2,fv2,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=1.3457290837237679e-13


Як бачимо, для заданої функції відхилення від її значень значень полінома швидко зменшується в нормі $\| \cdot \|_\infty$ із зростанням кількості вузлів. Як відомо (див.розділ ), при інтерполюванні розглянутими поліномами у випадку рівновіддалених вузлів так буде не для довільної функції. Це демонструє наступний приклад.

**Приклад 3.(example 8.1)** Інтерполювання функції $f(x)=(1+x^2)^{-1},\; x \in [-5,5]$  на **рівновіддалених вузлах**.

Виконаємо аналогічні кроки, що і в попередньому прикладі і розглянемо інтерполювання при зростанні кількості рівновіддалених вузлів:

In [20]:
def xv3(a,b,n):
    x=np.linspace(a,b,n+1)
    return x

In [21]:
def fv3(a,b,n):   
    x=np.linspace(a,b,n+1)
    f=1/(1+x*x)
    return f

In [22]:
a=-5
b=5

In [23]:
n = 5
m = 5
pN = N_interpolator(xv3,fv3,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=0.4326923076923076


In [24]:
n = 10
m = 10
pN = N_interpolator(xv3,fv3,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=1.9007638550703447


In [25]:
n = 20
m = 20
pN = N_interpolator(xv3,fv3,a,b,n,m,ng,fr=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

eps=56.63118692857639


Як бачимо, для заданої функції відхилення значень інтерполяційного полінома зростає поблизу кінців відрізка.

In [ ]:
plt.close('all')